## Ejercicio 2.2: Modelo de regresión logística que predice si un individuo tendrá una cita en el día de San Valentín o no.

**Enunciado**

Se tiene el siguiente conjunto de datos sintéticos creado para permitir a los aspirantes a científicos o analistas de datos llevar a cabo expreimentos predictivos mediante la construcción de un modelo de aprendizaje automático que puede predecir si un individuo obtendrá una cita de San Valentín o no.

El conjunto de datos tiene las siguientes características:

- Name: el nombre ficticio de una persona.
- Age: la edad ficticia de una persona.
- Gender: el género de  una persona.
- Income: lo que la persona gana anualmente como ingreso.
- Appearance Score: esta característica representa una puntuación numérica que indica cómo se percibe a los individuos en función de su apariencia física.
- Interest Score: esta característica representa una puntuación numérica que indica qué tan alineados o compatibles están los intereses de una persona con los de los demás.
- Confidence Score: esta característica representa una puntuación numérica que indica el nivel de confianza percibido de un individuo.
- Educational Status: esta característica representa el logro educativo o el nivel de educación de un individuo.
- Job Type: esta característica representa el tipo de trabajo o situación laboral de un individuo.
- Valentine Date: esta es una característica binaria utilizada para la clasificación binaria (0, 1), que indica si un individuo tiene una fecha de San Valentín (1) o no (0).

El conjunto de datos se encuentra en el archivo: ***ValentineDataset.csv***.

Utilizando un conjunto de datos proporcionado, realizar lo siguiente:

<ol type="a">
<li>Cargar y explorar los datos.</li>
<li>Hallar la matriz de correlación de los datos para determinar las variables más significativas a usar en el modelo.</li>
<li>Dividir los datos en 90% para entrenamiento y el resto para prueba.</li>
<li>Implementar un modelo de regresión logística utilizando Python y entrenarlo.</li>
<li>Graficar la función de pérdida obtenida durante entrenamiento y analizarla.</li>
<li>Evaluar el modelo utilizando la matriz de confusión y las métricas: precision, accuracy y recall.</li>
</ol>



### Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Cargar y explorar los datos

In [ ]:
df = pd.read_csv('ValentineDataset.csv')
df = df.dropna()
df.head(20)